In [1]:
!pip install Cartopy
!pip install pyqt5
!pip install Pillow

In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.figure_factory as ff
from matplotlib.animation import FuncAnimation
matplotlib.use('TkAgg')
import cartopy.crs as ccrs
import glob
from PIL import Image
BAD_FLAG = -1.e+34

c:\Users\Kalyan Ram\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [3]:
files_for_zonal=glob.glob("./Zonal/*.txt")
files_for_meridional=glob.glob("./Meridional/*.txt")
files_for_meridional.sort()
files_for_zonal.sort()
files_for_meridional,files_for_zonal

(['./Meridional\\Feb-06-2012.txt',
  './Meridional\\Feb-07-2012.txt',
  './Meridional\\Feb-08-2012.txt',
  './Meridional\\Feb-09-2012.txt',
  './Meridional\\Feb-10-2012.txt',
  './Meridional\\Feb-11-2012.txt',
  './Meridional\\Feb-12-2012.txt'],
 ['./Zonal\\Feb-06-2012.txt',
  './Zonal\\Feb-07-2012.txt',
  './Zonal\\Feb-08-2012.txt',
  './Zonal\\Feb-09-2012.txt',
  './Zonal\\Feb-10-2012.txt',
  './Zonal\\Feb-11-2012.txt',
  './Zonal\\Feb-12-2012.txt'])

In [4]:
dfzonallist=[]
dfmeridionallist=[]

In [5]:
def change(x):
    x=x.replace("S","")
    x="-"+x
    return x

for fil1,fil2 in zip(files_for_zonal,files_for_meridional):
    
    dfv=pd.read_table(fil1,header=6)
    dfu=pd.read_table(fil2,header=6)
    columnsspeed={}
    columnsindex={}

    for col in dfv.columns:
        newcol=col.replace(" ","")
        if 'E' in newcol: 
            finalcol=newcol.replace("E", "")
            columnsspeed[col]=finalcol
        elif 'W' in newcol: 
            finalcol=newcol.replace("W","")
            finalcol="-"+finalcol
            columnsspeed[col]=finalcol
        else:
            columnsspeed[col]=newcol
    
    
    dfv=dfv.rename(columns=columnsspeed)
    dfu=dfu.rename(columns=columnsspeed)

    dfv['']=dfv[''].apply(lambda x : x.replace("N","") if 'N' in x else x)
    dfv['']=dfv[''].apply(lambda x : change(x) if 'S' in x else x )
    dfu['']=dfu[''].apply(lambda x : x.replace("N","") if 'N' in x else x)
    dfu['']=dfu[''].apply(lambda x : change(x) if 'S' in x else x )
    dfv=dfv.set_index('')
    dfu=dfu.set_index('')
    dfu.index=dfu.index.astype(float)
    dfv.index=dfv.index.astype(float)
    dfv=dfv.replace(BAD_FLAG,np.nan) #np.nan
    dfu=dfu.replace(BAD_FLAG,np.nan)

    dfzonallist.append(dfv)
    dfmeridionallist.append(dfu)

dfzonallist,dfmeridionallist

([         0  0.5   1  1.5   2  2.5   3  3.5   4  4.5  ...  -4.5  -4  -3.5  -3  \
                                                       ...                       
  -89.75 NaN  NaN NaN  NaN NaN  NaN NaN  NaN NaN  NaN  ...   NaN NaN   NaN NaN   
  -89.25 NaN  NaN NaN  NaN NaN  NaN NaN  NaN NaN  NaN  ...   NaN NaN   NaN NaN   
  -88.75 NaN  NaN NaN  NaN NaN  NaN NaN  NaN NaN  NaN  ...   NaN NaN   NaN NaN   
  -88.25 NaN  NaN NaN  NaN NaN  NaN NaN  NaN NaN  NaN  ...   NaN NaN   NaN NaN   
  -87.75 NaN  NaN NaN  NaN NaN  NaN NaN  NaN NaN  NaN  ...   NaN NaN   NaN NaN   
  ...     ..  ...  ..  ...  ..  ...  ..  ...  ..  ...  ...   ...  ..   ...  ..   
   87.75 NaN  NaN NaN  NaN NaN  NaN NaN  NaN NaN  NaN  ...   NaN NaN   NaN NaN   
   88.25 NaN  NaN NaN  NaN NaN  NaN NaN  NaN NaN  NaN  ...   NaN NaN   NaN NaN   
   88.75 NaN  NaN NaN  NaN NaN  NaN NaN  NaN NaN  NaN  ...   NaN NaN   NaN NaN   
   89.25 NaN  NaN NaN  NaN NaN  NaN NaN  NaN NaN  NaN  ...   NaN NaN   NaN NaN   
   89.75 NaN  Na

In [6]:
def SameSizePlotQuiver(i):
    
    dfu=dfzonallist[i].iloc[120:150, 60:100]
    dfv=dfmeridionallist[i].iloc[120:150, 60:100]

    lat=np.array(dfu.columns.unique())
    lon=np.array(dfu.index)
    lat=lat.astype(np.float64)
    lon=lon.astype(np.float64)
    lat,lon=np.meshgrid(lat,lon)

    U=np.array(dfu)
    V=np.array(dfv)

    # Calculate vector magnitudes
    magnitude = np.sqrt(U**2 + V**2)

# Normalize vectors
    U /= magnitude
    V /= magnitude

    fig,ax=plt.subplots(figsize=(20,20))
    ax.xaxis.set_tick_params(labelbottom=False)
    ax.yaxis.set_tick_params(labelleft=False)

    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.coastlines()
    ax.gridlines(draw_labels=True)
    ax.set_title(files_for_meridional[i].split(".txt")[0][13:])
    quiv = plt.quiver(lat, lon, U, V,magnitude, cmap='Blues', pivot='middle',scale=40)
    
    cbar = plt.colorbar(quiv, ax=ax)
    cbar.set_label('Magnitude')
    
    plt.savefig("./Images1/"+files_for_meridional[i].split(".txt")[0][13:]+".png",bbox_inches='tight')
    return quiv,ax

In [7]:
for i in range(len(dfmeridionallist)):
    SameSizePlotQuiver(i)

In [8]:
files = glob.glob("./Images1/*.png")
image_array = []
for my_file in files:
    
    image = Image.open(my_file)
    image_array.append(image)

# print('image_arrays shape:', np.array(image_array).shape)
fig, ax = plt.subplots(figsize=(22,22))

ax.axis('off')  # Hide the axis
ax.set_frame_on(False)
# Set the initial image
im = ax.imshow(image_array[0],animated=True)

In [9]:
def update(i):
    im.set_array(image_array[i])
    return im,

In [10]:

animation= FuncAnimation(fig, update, frames=len(image_array),blit=True, interval=1000,repeat_delay=20)

animation.save("./Images1/animated_GMM.gif")

MovieWriter ffmpeg unavailable; using Pillow instead.
